In [3]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/fashionmnist/t10k-labels-idx1-ubyte
/kaggle/input/fashionmnist/t10k-images-idx3-ubyte
/kaggle/input/fashionmnist/fashion-mnist_test.csv
/kaggle/input/fashionmnist/fashion-mnist_train.csv
/kaggle/input/fashionmnist/train-labels-idx1-ubyte
/kaggle/input/fashionmnist/train-images-idx3-ubyte
/kaggle/input/digit-recognizer/sample_submission.csv
/kaggle/input/digit-recognizer/train.csv
/kaggle/input/digit-recognizer/test.csv
/kaggle/input/titanic/train.csv
/kaggle/input/titanic/test.csv
/kaggle/input/titanic/gender_submission.csv


In [4]:
!pip install torchvision

In [6]:
import torch,torchvision
from torchvision import datasets
from torch.utils.data import DataLoader
from torchvision.transforms import ToTensor
import torch.nn as nn

In [7]:
train_dataset=datasets.MNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),
)
test_dataset=datasets.MNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
)

100%|██████████| 9912422/9912422 [00:00<00:00, 116661507.12it/s]


Extracting data/MNIST/raw/train-images-idx3-ubyte.gz to data/MNIST/raw



100%|██████████| 28881/28881 [00:00<00:00, 31869427.47it/s]


Extracting data/MNIST/raw/train-labels-idx1-ubyte.gz to data/MNIST/raw



100%|██████████| 1648877/1648877 [00:00<00:00, 38196471.89it/s]


Extracting data/MNIST/raw/t10k-images-idx3-ubyte.gz to data/MNIST/raw



100%|██████████| 4542/4542 [00:00<00:00, 10818017.47it/s]


Extracting data/MNIST/raw/t10k-labels-idx1-ubyte.gz to data/MNIST/raw



In [9]:
batch_size=64

train_dl=DataLoader(
    dataset=train_dataset,
    batch_size=batch_size,
    shuffle=True
)
test_dl=DataLoader(
    dataset=test_dataset,
    batch_size=batch_size, 
    shuffle=True
)

In [ ]:
for i,j in train_dl:
    print(i, j)
    break

In [163]:
class mymodel(nn.Module):
    def __init__(self):
        super().__init__()
        self.inh1=nn.Linear(in_features=784, out_features=512)
        self.relu=nn.ReLU()
        self.bn1=nn.BatchNorm1d(num_features=512)
        self.h2=nn.Linear(in_features=512, out_features=256)
        self.bn2=nn.BatchNorm1d(num_features=256)
        self.h3=nn.Linear(in_features=256, out_features=128)
        self.bn3=nn.BatchNorm1d(num_features=128)
        self.h4=nn.Linear(in_features=128, out_features=64)
        self.bn4=nn.BatchNorm1d(num_features=64)
        self.h5=nn.Linear(in_features=64, out_features=32)
        self.bn5=nn.BatchNorm1d(num_features=32)
        self.output=nn.Linear(in_features=32, out_features=10)
        self.bn6=nn.BatchNorm1d(num_features=10)
        self.flat=nn.Flatten()
        self.d1=nn.Linear(in_features=512*512, out_features=10)
        
    def forward(self, x):
        x=self.inh1(x)
        x=self.bn1(x)
        x=self.relu(x)
        x=self.h2(x)
        x=self.bn2(x)
        x=self.relu(x)
        x=self.h3(x)
        x=self.bn3(x)
        x=self.relu(x)
        x=self.h4(x)
        x=self.bn4(x)
        x=self.relu(x)
        x=self.h5(x)
        x=self.bn5(x)
        x=self.relu(x)
        x=self.output(x)
        output=self.bn6(x)
        return output
    
def train_one_epoch(dataloader, model, loss_fn, optimizer):
    model.train()
    track_loss=0
    num_correct=0
    for i,(img, labels) in enumerate(dataloader):
        imgs=torch.reshape(img, shape=[-1, 784])
        labels=labels
        pred=model(imgs)
#         print(pred.shape, labels.shape)
        loss=loss_fn(pred, labels)
        track_loss+=loss.item()
        num_correct+=(torch.argmax(pred, dim=1)==labels).type(torch.float).sum().item()
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        
    if i%100==0:
        running_loss=round(track_loss/(i+1),2)
        running_acc=round((num_correct/((i+1)*batch_size))*100,2)
        print("Batch:", i+1, "/",len(dataloader), "Running Loss:",running_loss, "Running Accuracy:",running_acc)
    epoch_loss=track_loss/len(dataloader) #can be slightly inaccurate
    epoch_acc=(num_correct/len(dataloader.dataset))*100    
    return round(epoch_loss,2), round(epoch_acc,2)

In [1]:
model=mymodel()
model=model
loss_fn=nn.CrossEntropyLoss()
lr=0.001
#optimizer=torch.optim.SGD(params=model.parameters(), lr=lr)
optimizer=torch.optim.Adam(params=model.parameters(), lr=lr)
n_epochs=5

for i in range(n_epochs):
    print("Epoch No:",i+1)
    train_epoch_loss, train_epoch_acc=train_one_epoch(train_dl,model,loss_fn,optimizer)
#     print(j)
#     val_epoch_loss, val_epoch_acc=eval_one_epoch(test_dl,model,loss_fn)
    print("Training:", "Epoch Loss:", train_epoch_loss, "Epoch Accuracy:", train_epoch_acc)
#     print("Inference:", "Epoch Loss:", val_epoch_loss, "Epoch Accuracy:", val_epoch_acc)
    print("--------------------------------------------------")


NameError: name 'mymodel' is not defined

In [186]:
import torch
import torch.nn as nn

import torch
import torch.nn as nn

class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3)#28-2
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3)#13-2
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3)#5-2
        self.pool = nn.MaxPool2d(2, 2)
        self.out = nn.ReLU()
        self.dropout=nn.Dropout(0.25)
        self.fc2 = nn.Linear(128, 10)
        self.fl=nn.Flatten()
    def forward(self, x):
        x = self.dropout(self.pool(self.out(self.conv1(x))))
        x = self.dropout(self.pool(self.out(self.conv2(x))))
        x = self.dropout(self.pool(self.out(self.conv3(x))))
        x = self.fl(x)
        x = self.fc2(x)
        return x

# Create an instance of the CNN model
def train_one_epoch2(dataloader, model, loss_fn, optimizer):
    model.train()
    track_loss=0
    num_correct=0
    for j,(img, labels) in enumerate(dataloader):
        labels=labels
        pred=model(img)
        loss=loss_fn(pred, labels)
        track_loss+=loss.item()
        num_correct+=(torch.argmax(pred, dim=1)==labels).type(torch.float).sum().item()
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        
    if i%100==0:
        running_loss=round(track_loss/(i+1),2)
        running_acc=round((num_correct/((i+1)*batch_size))*100,2)
        print("Batch:", i+1, "/",len(dataloader), "Running Loss:",running_loss, "Running Accuracy:",running_acc)
    epoch_loss=track_loss/len(dataloader) #can be slightly inaccurate
    epoch_acc=(num_correct/len(dataloader.dataset))*100    
    return round(epoch_loss,2), round(epoch_acc,2)

def eval_one(dataloader, model):
    track_loss=0
    num_correct=0
    for j,(img, labels) in enumerate(dataloader):
        labels=labels
        pred=model(img)
        loss=loss_fn(pred, labels)
        track_loss+=loss.item()
        num_correct+=(torch.argmax(pred, dim=1)==labels).type(torch.float).sum().item()
        
    if i%100==0:
        running_loss=round(track_loss/(i+1),2)
        running_acc=round((num_correct/((i+1)*batch_size))*100,2)
        print("Batch:", i+1, "/",len(dataloader), "Validation Loss:",running_loss, "Validation Accuracy:",running_acc)
    epoch_loss=track_loss/len(dataloader) #can be slightly inaccurate
    epoch_acc=(num_correct/len(dataloader.dataset))*100    
    return round(epoch_loss,2), round(epoch_acc,2)

In [190]:
model=CNN()
loss_fn=nn.CrossEntropyLoss()
lr=0.001
#optimizer=torch.optim.SGD(params=model.parameters(), lr=lr)
optimizer=torch.optim.Adam(params=model.parameters(), lr=lr)
n_epochs=5

for i in range(n_epochs):
    print("Epoch No:",i+1)
    train_epoch_loss, train_epoch_acc=train_one_epoch2(train_dl,model,loss_fn,optimizer)
#     print(j)
    val_epoch_loss, val_epoch_acc=eval_one_epoch(test_dl,model,loss_fn)
    print("Training:", "Epoch Loss:", train_epoch_loss, "Epoch Accuracy:", train_epoch_acc)
    print("Validation:", "Epoch Loss:", val_epoch_loss, "Epoch Accuracy:", val_epoch_acc)
    print("--------------------------------------------------")


Epoch No: 1
Batch: 1 / 938 Running Loss: 359.62 Running Accuracy: 82357.81
Batch: 1 / 157 Validation Loss: 22.08 Validation Accuracy: 14965.62
Training: Epoch Loss: 0.38 Epoch Accuracy: 87.85
Validation: Epoch Loss: 0.14 Epoch Accuracy: 95.78
--------------------------------------------------
Epoch No: 2
Training: Epoch Loss: 0.14 Epoch Accuracy: 95.83
Validation: Epoch Loss: 0.12 Epoch Accuracy: 96.68
--------------------------------------------------
Epoch No: 3
Training: Epoch Loss: 0.11 Epoch Accuracy: 96.66
Validation: Epoch Loss: 0.1 Epoch Accuracy: 96.54
--------------------------------------------------
Epoch No: 4
Training: Epoch Loss: 0.09 Epoch Accuracy: 97.24
Validation: Epoch Loss: 0.1 Epoch Accuracy: 97.3
--------------------------------------------------
Epoch No: 5
Training: Epoch Loss: 0.08 Epoch Accuracy: 97.56
Validation: Epoch Loss: 0.08 Epoch Accuracy: 97.69
--------------------------------------------------
